In [1]:
import spacy
import pandas as pd
import sklearn
import numpy as np
from scipy.spatial.distance import cosine
import random
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [2]:
en_nlp = spacy.en.English()

In [3]:
def get_vec(sentence):
    vec = np.zeros(300)
    for word in sentence:
        if word in wordvecs:
            vec += wordvecs[word]
    return vec
    
    

In [4]:
def spacy_get_vec(sentence):
    vec = np.zeros(300)
    doc = en_nlp((sentence))
    for word in doc:
        vec += word.vector 
    return vec

In [5]:
lines = open('./intents.txt').readlines()

In [6]:
vecs = []
intents = []
for line in lines:
    tokens = line.split(',')
    sentence = tokens[0]
    intent = tokens[1]
    if intent[-1] == '\n':
        intent = intent[:-1]
    vecs.append(spacy_get_vec(sentence))
    intents.append(intent)

In [7]:
df = pd.DataFrame(vecs, columns=['vec_%d' % i for i in range(300)])

In [8]:
df['intents'] = intents

In [9]:
df.intents = df.intents.astype('category')

In [10]:
X = df.iloc[:, :-1].values

In [11]:
y = df.iloc[:,-1:].values.ravel()

In [12]:
from sklearn.linear_model import LogisticRegression
logit_model = LogisticRegression()
logit_model.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
logit_model.score(X, y)

0.95454545454545459

In [14]:
from sklearn.ensemble import GradientBoostingClassifier

In [15]:
gradboost = GradientBoostingClassifier(n_estimators=500, max_depth=25, max_features='sqrt')

In [16]:
gradboost.fit(X, y)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=25,
              max_features='sqrt', max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=500, presort='auto', random_state=None,
              subsample=1.0, verbose=0, warm_start=False)

In [17]:
gradboost.score(X, y)

1.0

In [18]:
sent = 'that looks good'
print(gradboost.predict(spacy_get_vec(sent)))
print(gradboost.predict_proba(spacy_get_vec(sent)))

['greeting']
[[ 0.06126139  0.65733628  0.01950067  0.26190167]]


In [26]:
sent = 'can you tell the forecast for tomorrow'
print(gradboost.predict(spacy_get_vec(sent)))
print(gradboost.predict_proba(spacy_get_vec(sent)))

['weather']
[[ 0.01439606  0.00439994  0.31790727  0.66329673]]


In [20]:
gradboost.classes_

array(['goodbye', 'greeting', 'travel', 'weather'], dtype=object)

In [21]:
from sklearn.neural_network import MLPClassifier
nn = MLPClassifier(hidden_layer_sizes=(300, 128, 64, 16, 4), max_iter=1000)
nn.fit(X, y)
nn.score(X, y)

1.0

In [22]:
sent = 'how does the sky look for tomorrow'
print(nn.predict(spacy_get_vec(sent)))
print(nn.predict_proba(spacy_get_vec(sent)))

['weather']
[[  6.98517995e-07   2.20998801e-37   1.24367430e-03   9.98755627e-01]]


In [23]:
logit_model.predict_proba(spacy_get_vec('is there a hailstorm outside').reshape(1,-1))

array([[ 0.08015599,  0.09077133,  0.11935961,  0.70971306]])

In [24]:
from sklearn.externals import joblib
joblib.dump(gradboost, 'intent.pkl')

['intent.pkl']